In [55]:
import pandas as pd
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import re
import ast

In [ ]:
#load data
df = pd.read_csv("data/original_data/plot_summaries.txt", delimiter = "\t", names=["id", "text"])
df.head()

,id,text
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [ ]:
def clean_text(df):
    """
    This function tokenize, removes stop words, removes headers of text and applies stemming to the words to dataframe with column 'text'.
    """
    #remove header contained in brackets {}
    for text in df['text']:
        re.sub(r"\{([^}]*)\}", "", text)

    for p in string.punctuation:
        df['text'] = df['text'].str.replace(p, "")
    df['text'] = df['text'].str.split(" ")

    stop_words = set(stopwords.words("english"))
    
    stemmer = PorterStemmer()

    def remove_stopwords_and_stem(text):
        """
        This function removes stopwords and stems the words
        It also removes empty strings created by the removal of punctuation
        """
        return [stemmer.stem(word.lower()) for word in text if ((word.lower() not in stop_words) and (len(word)>1) and (word.lower()))]

        
    df['text'] = df['text'].apply(remove_stopwords_and_stem)
    df.to_csv('stemmed_movie_summaries.csv')

    return df

In [ ]:
#cleaning
df = clean_text(df)
df.head(10)

,id,text
0,23890098,"[shlykov, hardwork, taxi, driver, lyosha, saxo..."
1,31186339,"[nation, panem, consist, wealthi, capitol, twe..."
2,20663735,"[pooval, induchoodan, sentenc, six, year, pris..."
3,2231378,"[lemon, drop, kid, new, york, citi, swindler, ..."
4,595909,"[seventhday, adventist, church, pastor, michae..."
5,5272176,"[presid, way, give, speech, travel, man, show,..."
6,1952976,"[plot, film, open, 1974, young, girl, dahlia, ..."
7,24225279,"[stori, begin, hannah, young, jewish, teen, co..."
8,2462689,"[infuri, told, write, one, final, column, laid..."
9,20532852,"[line, peopl, drool, window, shop, market, but..."


In [ ]:
#load theme
death_synonyms = [ "killed", "murdered", "assassinated", "slain", "shot", "stabbed", "executed", "died", "perished", "succumbed", "bled", "decapitated", "strangled", "blown", "electrocuted", "drowned", "burned", "poisoned", "crushed", "impaled", "asphyxiated", "bludgeoned", "suffocated", "choked", "eviscerated", "slaughtered", "disintegrated", "torn", "gutted", "hanged", "gassed", "slashed", "ripped", "cut", "disemboweled", "blasted", "tortured", "beheaded", "smashed", "mauled", "knifed", "eaten", "struck", "dismembered", "skinned", "hacked", "pummeled", "pierced", "overpowered", "collapsed", "punched", "squashed", "sliced", "flayed", "stomped", "shattered", "flattened", "scalded", "severed", "hit", "pinned", "decayed", "blasted", "bludgeoned", "throttled", "attacked", "electrocuted", "incinerated" ]

In [ ]:
# clean theme words
death_df = pd.DataFrame({'text':death_synonyms})
death_df = clean_text(death_df)
death_df['text'] = death_df['text'].apply(lambda x: x[0])
death_df.head()

,text
0,kill
1,murder
2,assassin
3,slain
4,shot


In [ ]:
def find_theme(text_df, theme_df):
    """
    Returns the normalized vector with  number of words in theme_df that each text in text_df contains
    """
    encoding = np.zeros(len(text_df))
    for i, text in enumerate(text_df['text']):
        for _, word in enumerate(theme_df['text'].values):
            if word in text:
                encoding[i] += 1
    max_val = np.max(encoding)
    encoding = encoding/max_val
    return encoding

In [ ]:
#encoding
encoding = find_theme(df, death_df)
encoding

array([0.  , 0.3 , 0.15, ..., 0.  , 0.05, 0.05])